In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pyuvdata
import sys
sys.path.append("/Users/ruby/Astro/rlb_LWA/beam_manipulation")
import jones_to_mueller
import importlib
importlib.reload(jones_to_mueller)

<module 'jones_to_mueller' from '/Users/ruby/Astro/rlb_LWA/beam_manipulation/jones_to_mueller.py'>

In [2]:
beam_nm = pyuvdata.UVBeam()
beam_nm.read("/Users/ruby/Astro/LWA_beams/LWA_10to100.beamfits")
beam_nm.peak_normalize()

In [3]:
beam_dw = jones_to_mueller.read_beam_txt_file('/Users/ruby/Astro/LWA_beams/DW_beamquadranttable20151110.txt', header_line=6)
beam_dw.peak_normalize()

In [4]:
beam_nm.freq_array

array([[1.0e+07, 1.1e+07, 1.2e+07, 1.3e+07, 1.4e+07, 1.5e+07, 1.6e+07,
        1.7e+07, 1.8e+07, 1.9e+07, 2.0e+07, 2.1e+07, 2.2e+07, 2.3e+07,
        2.4e+07, 2.5e+07, 2.6e+07, 2.7e+07, 2.8e+07, 2.9e+07, 3.0e+07,
        3.1e+07, 3.2e+07, 3.3e+07, 3.4e+07, 3.5e+07, 3.6e+07, 3.7e+07,
        3.8e+07, 3.9e+07, 4.0e+07, 4.1e+07, 4.2e+07, 4.3e+07, 4.4e+07,
        4.5e+07, 4.6e+07, 4.7e+07, 4.8e+07, 4.9e+07, 5.0e+07, 5.1e+07,
        5.2e+07, 5.3e+07, 5.4e+07, 5.5e+07, 5.6e+07, 5.7e+07, 5.8e+07,
        5.9e+07, 6.0e+07, 6.1e+07, 6.2e+07, 6.3e+07, 6.4e+07, 6.5e+07,
        6.6e+07, 6.7e+07, 6.8e+07, 6.9e+07, 7.0e+07, 7.1e+07, 7.2e+07,
        7.3e+07, 7.4e+07, 7.5e+07, 7.6e+07, 7.7e+07, 7.8e+07, 7.9e+07,
        8.0e+07, 8.1e+07, 8.2e+07, 8.3e+07, 8.4e+07, 8.5e+07, 8.6e+07,
        8.7e+07, 8.8e+07, 8.9e+07, 9.0e+07, 9.1e+07, 9.2e+07, 9.3e+07,
        9.4e+07, 9.5e+07, 9.6e+07, 9.7e+07, 9.8e+07, 9.9e+07, 1.0e+08]])

In [5]:
beam_dw.freq_array[0,:]

array([20000000., 30000000., 40000000., 50000000., 60000000., 70000000.,
       80000000.])

In [9]:
use_cmap = matplotlib.cm.get_cmap("Spectral").copy()
use_cmap.set_bad(color="whitesmoke")
vmin=-1
vmax=1

for feed_ind in range(2):
    for freq in beam_nm.freq_array[0,:]:
    #for freq in [7e7]:
        savepath = f"/Users/ruby/Astro/compare_NM_DW_beams_Jun2023/feed{(['X','Y'])[feed_ind]}_freq{int(freq/1e6)}.png"
        #savepath = None

        fig, ax = plt.subplots(
            nrows=4, ncols=2, subplot_kw=dict(projection="polar"), figsize=(6, 9)
        )
        for real_imag_ind in range(2):
            for beam_ind, beam in enumerate([beam_nm, beam_dw]):

                use_freq = beam.freq_array[0, np.argmin(np.abs(beam.freq_array[0, :] - freq))]
                use_beam = beam.select(frequencies=[use_freq], inplace=False)
                az_axis = np.degrees(use_beam.axis1_array)
                za_axis = np.degrees(use_beam.axis2_array)

                #horizon cut
                use_za_inds = np.where(za_axis < 90.)[0]
                use_beam.select(axis2_inds=use_za_inds, inplace=True)
                za_axis = za_axis[use_za_inds]

                if real_imag_ind == 0:
                    plot_jones_vals = np.real(use_beam.data_array)
                else:
                    plot_jones_vals = np.imag(use_beam.data_array)

                za_vals, az_vals = np.meshgrid(za_axis, az_axis)

                for pol_ind, pol_name in enumerate(["Az", "ZA"]):
                    contourplot = jones_to_mueller.make_polar_contour_plot(
                        ax[real_imag_ind + 2*pol_ind, beam_ind],
                        (plot_jones_vals[pol_ind, 0, feed_ind, 0, :, :]).T,
                        np.radians(az_vals),
                        za_vals,
                        vmin=vmin,
                        vmax=vmax,
                    )
                    fig.colorbar(contourplot, ax=ax[real_imag_ind + 2*pol_ind, beam_ind])

                    subtitle = f"{(['NM', 'DW'])[beam_ind]} {int(use_freq/1e6)}MHz, {(['Real','Imag'])[real_imag_ind]}, {pol_name}"
                    ax[real_imag_ind + 2*pol_ind, beam_ind].set_title(subtitle)

        fig.suptitle(f"Feed {(['X','Y'])[feed_ind]}")
        fig.tight_layout()
        if savepath is None:
            plt.show()
            plt.close()
        else:
            plt.savefig(savepath, dpi=300)
            plt.close()